In [2]:
import psycopg2
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

host="13.43.84.151"
dbname="postgres"
user="enpei.xia.23@ucl.ac.uk"
password="uPyOwz"
port="5432"
schema="schema_enpeixia23uclacuk"

In [3]:
conn = psycopg2.connect (
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)
print("Database connection established")

Database connection established


In [4]:
# fetch the java driver for postgress to allow Spark to connect to postgress
!curl -o postgresql-42.3.2.jar https://jdbc.postgresql.org/download/postgresql-42.3.2.jar
# uses the curl command to download the PostgreSQL JDBC driver from the official site.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1015k  100 1015k    0     0   810k      0  0:00:01  0:00:01 --:--:--  810k


In [5]:
import os
current_dir = os.getcwd() # assigns the current working directory to the variable current_dir
print(current_dir)

jar_location = current_dir + "/postgresql-42.3.2.jar"
# Constructs the file path for the PostgreSQL JDBC driver JAR file by appending the filename to the current directory path
jar_location

/home/jovyan/dataengineer


'/home/jovyan/dataengineer/postgresql-42.3.2.jar'

In [6]:
from pyspark.sql import SparkSession
# Initialize Spark Session
spark = SparkSession.builder \
    .config("spark.driver.extraClassPath", jar_location)\
    .appName("Enhanced Data Warehouse ETL") \
    .getOrCreate()
# Sets the configuration property spark.driver.extraClassPath to the path of the JDBC driver.
# This tells Spark where to find the driver for the database connection.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/09 11:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The data from the **following 5 tables** (Upper Value Quartile, Median Value Quartile,  Lower Value Quartile, and Number of houses by price range) are sourced from the **American Community Survey (ACS) for the year 2022**. This survey is conducted by the U.S. Census Bereau, which provides statistics and estimates about the U.S. population and housing characteristics, covering more than 40 topics. The ACS is crucial for establishing government policies and helping with the decision-making processes. 

It is important to note that the ACS data is based on a sample and thus subject to *sampling variability*. The degree of uncertainty for an estimate is represented by the 90 percent margin of error, which is the confidence interval within which the true value is expected to lie with a 90% probability. For simplification reasons, we have not included the margin of errors in the following tables. Additionally, the ACS estimates might be subject to *nonsampling error*, and the effect of this type of error is not represented in these tables. 

For more information on this data source and its methodology, we recommend visiting the official website of the U.S. Census of Bureau or ACS website. Supporting documentation such as data accuracy, hypothesis testing and full tables can be found on the Technical Documentation section of the ACS website. 

Let’s import the **first CSV table from the ACS**, which contains the upper quartile estimates for house prices in each state. This table will be loaded into a Spark dataframe, followed by data cleaning using Spark to improve data readability and usability. Below, we display the table both prior to and after the cleaning process for comparison. 

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, lit, lead

# Read the CSV file into a Spark DataFrame
upper_value_df = spark.read.csv("upper_value.csv", header=True, inferSchema=True)
# Show the CSV file before cleaning 
upper_value_df.show()

# Define a window spec to look ahead by one row
windowSpec = Window.orderBy(lit(0))

# Shift the 'Upper value quartile (dollars)' column up by one row
upper_value_df = upper_value_df.withColumn("shifted_value", lag("Upper value quartile (dollars)").over(windowSpec))

# Filter out the 'Estimate' rows and keep the rows with state names
# Then drop the original 'Upper value quartile (dollars)' column
# Lastly, remove rows where the 'shifted_value' is null
upper_value_clean = upper_value_df.filter(~col("Label (Grouping)").contains("Estimate")) \
             .drop("Upper value quartile (dollars)") \
             .filter(col("shifted_value").isNotNull()) \
             .withColumnRenamed("shifted_value", "Upper value quartile (dollars)") \
             .withColumnRenamed("Label (Grouping)", "State")

# Show the DataFrame after cleaning 
upper_value_clean.show()

+--------------------+------------------------------+
|    Label (Grouping)|Upper value quartile (dollars)|
+--------------------+------------------------------+
|             Alabama|                          NULL|
|            Estimate|                       331,200|
|              Alaska|                          NULL|
|            Estimate|                       455,200|
|             Arizona|                          NULL|
|            Estimate|                       602,700|
|            Arkansas|                          NULL|
|            Estimate|                       290,700|
|          California|                          NULL|
|            Estimate|                     1,083,100|
|            Colorado|                          NULL|
|            Estimate|                       737,500|
|         Connecticut|                          NULL|
|            Estimate|                       504,900|
|            Delaware|                          NULL|
|            Estimate|      

24/03/09 11:54:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+------------------------------+
|               State|Upper value quartile (dollars)|
+--------------------+------------------------------+
|              Alaska|                       331,200|
|             Arizona|                       455,200|
|            Arkansas|                       602,700|
|          California|                       290,700|
|            Colorado|                     1,083,100|
|         Connecticut|                       737,500|
|            Delaware|                       504,900|
|District of Columbia|                       468,200|
|             Florida|                     1,057,600|
|             Georgia|                       524,200|
|              Hawaii|                       450,200|
|               Idaho|                     1,133,500|
|            Illinois|                       638,200|
|             Indiana|                       383,500|
|                Iowa|                       320,100|
|              Kansas|      

Now let’s import the **second and third csv table from ACS**, which contains the median quartile and lower quartile estimates for house prices in each state. We will then perform the same cleaning steps for these tables. 

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window

# Read the CSV file into a Spark DataFrame
median_value_df = spark.read.csv("median_value.csv", header=True, inferSchema=True)
median_value_df.show()

# Define a window spec to look ahead by one row
windowSpec = Window.orderBy(lit(0))

# Shift the 'Median value (dollars)' column up by one row
median_value_df = median_value_df.withColumn("shifted_median_value", lag("Median value (dollars)").over(windowSpec))

# Filter out the 'Estimate' rows and keep the rows with state names
# Then drop the original 'Median value (dollars)' column
# Lastly, remove rows where the 'shifted_median_value' is null
median_value_clean = median_value_df.filter(~col("Label (Grouping)").contains("Estimate")) \
                                    .drop("Median value (dollars)") \
                                    .filter(col("shifted_median_value").isNotNull()) \
                                    .withColumnRenamed("shifted_median_value", "Median value (dollars)") \
                                    .withColumnRenamed("Label (Grouping)", "State")

# Show the cleaned DataFrame
median_value_clean.show()

+--------------------+----------------------+
|    Label (Grouping)|Median value (dollars)|
+--------------------+----------------------+
|             Alabama|                  NULL|
|            Estimate|               200,900|
|              Alaska|                  NULL|
|            Estimate|               336,900|
|             Arizona|                  NULL|
|            Estimate|               402,800|
|            Arkansas|                  NULL|
|            Estimate|               179,800|
|          California|                  NULL|
|            Estimate|               715,900|
|            Colorado|                  NULL|
|            Estimate|               531,100|
|         Connecticut|                  NULL|
|            Estimate|               347,200|
|            Delaware|                  NULL|
|            Estimate|               337,200|
|District of Columbia|                  NULL|
|            Estimate|               711,100|
|             Florida|            

24/03/09 11:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------------------+
|               State|Median value (dollars)|
+--------------------+----------------------+
|              Alaska|               200,900|
|             Arizona|               336,900|
|            Arkansas|               402,800|
|          California|               179,800|
|            Colorado|               715,900|
|         Connecticut|               531,100|
|            Delaware|               347,200|
|District of Columbia|               337,200|
|             Florida|               711,100|
|             Georgia|               354,100|
|              Hawaii|               297,400|
|               Idaho|               820,100|
|            Illinois|               432,500|
|             Indiana|               251,600|
|                Iowa|               208,700|
|              Kansas|               194,600|
|            Kentucky|               206,600|
|           Louisiana|               196,300|
|               Maine|            

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag
from pyspark.sql.window import Window


# Read the CSV file into a Spark DataFrame
lower_value_df = spark.read.csv('lower_value.csv', header=True, inferSchema=True)

# Show the initial state of the DataFrame
lower_value_df.show()

# Define a window spec to look ahead by one row
windowSpec = Window.orderBy(lit(0))

# Shift the 'Lower value (dollars)' column up by one row
lower_value_df = lower_value_df.withColumn("shifted_lower_value", lag("Lower value quartile (dollars)").over(windowSpec))

# Filter out the 'Estimate' rows and keep the rows which have the state names 
# Next, drop the original 'Lower value (dollars)' column
# Lastly, remove rows where the 'shifted_lower_value' is null
lower_value_clean = lower_value_df.filter(~col("Label (Grouping)").contains("Estimate")) \
                                  .drop("Lower value quartile (dollars)") \
                                  .filter(col("shifted_lower_value").isNotNull()) \
                                  .withColumnRenamed("shifted_lower_value", "Lower value quartile (dollars)") \
                                  .withColumnRenamed("Label (Grouping)", "State")

# Show the cleaned DataFrame
lower_value_clean.show()

+--------------------+------------------------------+
|    Label (Grouping)|Lower value quartile (dollars)|
+--------------------+------------------------------+
|             Alabama|                          NULL|
|            Estimate|                       105,000|
|              Alaska|                          NULL|
|            Estimate|                       232,200|
|             Arizona|                          NULL|
|            Estimate|                       262,500|
|            Arkansas|                          NULL|
|            Estimate|                        98,300|
|          California|                          NULL|
|            Estimate|                       460,900|
|            Colorado|                          NULL|
|            Estimate|                       366,900|
|         Connecticut|                          NULL|
|            Estimate|                       240,600|
|            Delaware|                          NULL|
|            Estimate|      

24/03/09 11:54:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+------------------------------+
|               State|Lower value quartile (dollars)|
+--------------------+------------------------------+
|              Alaska|                       105,000|
|             Arizona|                       232,200|
|            Arkansas|                       262,500|
|          California|                        98,300|
|            Colorado|                       460,900|
|         Connecticut|                       366,900|
|            Delaware|                       240,600|
|District of Columbia|                       228,600|
|             Florida|                       468,200|
|             Georgia|                       222,500|
|              Hawaii|                       175,300|
|               Idaho|                       513,300|
|            Illinois|                       293,700|
|             Indiana|                       148,100|
|                Iowa|                       126,400|
|              Kansas|      

In [10]:
# JDBC URL
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema={schema}"

# Connection properties
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

# Current directory and JDBC driver location
current_dir = os.getcwd()
jar_location = os.path.join(current_dir, "postgresql-42.3.2.jar")

# Write the cleaned median value DataFrame to PostgreSQL
median_value_clean.write.jdbc(url=url, table="median_value_table", mode="overwrite", properties=properties)

# Write the cleaned upper value DataFrame to PostgreSQL
upper_value_clean.write.jdbc(url=url, table="upper_value_table", mode="overwrite", properties=properties)

# Write the cleaned lower value DataFrame to PostgreSQL
lower_value_clean.write.jdbc(url=url, table="lower_value_table", mode="overwrite", properties=properties)

24/03/09 11:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 1

We can execute a simple SQL query to **combine the tables** of upper, median, and lower quartile estimates using state names as the key. After querying, this consolidated Spark dataframe can then be stored in our Postgres database.

In [11]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Data Joining").getOrCreate()

# Given the `upper_value_clean`, `median_value_clean`, and `lower_value_clean` data frame we already defined above
# Let's register these DataFrames as temporary views
upper_value_clean.createOrReplaceTempView("upper_value_view")
median_value_clean.createOrReplaceTempView("median_value_view")
lower_value_clean.createOrReplaceTempView("lower_value_view")

# Perform the SQL join query using state as the key, combining the upper,median,lower quartile values together 
join_query = """
SELECT 
  u.State, 
  u.`Upper value quartile (dollars)` AS Upper_Value,
  m.`Median value (dollars)` AS Median_Value,
  l.`Lower value quartile (dollars)` AS Lower_Value
FROM 
  upper_value_view u
JOIN 
  median_value_view m 
ON 
  u.State = m.State
JOIN 
  lower_value_view l
ON 
  u.State = l.State
"""

# Execute the query
combined_quartile_df = spark.sql(join_query)

# Show the results
combined_quartile_df.show()

# Write the cleaned quartile value DataFrame to PostgreSQL
combined_quartile_df.write.jdbc(url=url, table="combined_value_quartile", mode="overwrite", properties=properties)

24/03/09 11:54:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/03/09 11:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:30 WARN WindowExec: No Partition Defined

+--------------------+-----------+------------+-----------+
|               State|Upper_Value|Median_Value|Lower_Value|
+--------------------+-----------+------------+-----------+
|              Alaska|    331,200|     200,900|    105,000|
|             Arizona|    455,200|     336,900|    232,200|
|            Arkansas|    602,700|     402,800|    262,500|
|          California|    290,700|     179,800|     98,300|
|            Colorado|  1,083,100|     715,900|    460,900|
|         Connecticut|    737,500|     531,100|    366,900|
|            Delaware|    504,900|     347,200|    240,600|
|District of Columbia|    468,200|     337,200|    228,600|
|             Florida|  1,057,600|     711,100|    468,200|
|             Georgia|    524,200|     354,100|    222,500|
|              Hawaii|    450,200|     297,400|    175,300|
|               Idaho|  1,133,500|     820,100|    513,300|
|            Illinois|    638,200|     432,500|    293,700|
|             Indiana|    383,500|     2

24/03/09 11:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 1

Next we’ll import the **fourth csv table from ACS**, detailing the estimated number of houses in each state, categorized by price ranges, ranging from less than 50,000 USD to more than 1,000,000 USD. Given that this table is larger and more complicated, we utilize pandas for cleaning and then convert the cleaned pandas dataframe into a Spark dataframe. Finally, we store the spark dataframe in the Postgres database. 

In [12]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file_path = 'num_of_houses_by_price_range.csv'
df = pd.read_csv(csv_file_path)
df.head()

# Now we can do some data cleaning: 
# Rename the first column name to "State"
df.rename(columns={df.columns[0]: 'State'}, inplace=True)

# Remove the "Total:!!" prefix from the column names starting from the third column
df.columns = [col if idx < 2 else col.replace('Total:!!', '') for idx, col in enumerate(df.columns)]

# Shift all the data points one step upward in columns other than 'State'
for column in df.columns[1:]: #  iterate over each column, except 'State', and shift them
    df[column] = df[column].shift(-1)

# Drop all rows which contains NaN value 
df_cleaned = df.dropna()

# Display the cleaned DataFrame
df_cleaned.head()

# Convert the pandas DataFrame to the Spark DataFrame
spark_df_num_of_houses = spark.createDataFrame(df_cleaned)
spark_df_num_of_houses.show()

# Write the Spark DataFrame to the PostgreSQL database
spark_df_num_of_houses.write.jdbc(url=url, table="num_of_houses_by_price_range", mode='overwrite', properties=properties)


,Label (Grouping),Total:,"Total:!!Less than $50,000","Total:!!$50,000 to $99,999","Total:!!$100,000 to $149,999","Total:!!$150,000 to $199,999","Total:!!$200,000 to $299,999","Total:!!$300,000 to $499,999","Total:!!$500,000 to $999,999","Total:!!$1,000,000 or more"
0,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Estimate,"1,416,333","147,494","186,145","179,651","192,181","298,254","278,735","115,476","18,397"
2,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Estimate,"181,586","10,015","6,103","7,964","11,081","38,637","75,646","29,463","2,677"
4,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,State,Total:,"Less than $50,000","$50,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 to $299,999","$300,000 to $499,999","$500,000 to $999,999","$1,000,000 or more"
0,Alabama,"1,416,333","147,494","186,145","179,651","192,181","298,254","278,735","115,476","18,397"
2,Alaska,"181,586","10,015","6,103","7,964","11,081","38,637","75,646","29,463","2,677"
4,Arizona,"1,921,746","115,424","67,678","57,890","80,092","265,323","692,276","541,367","101,696"
6,Arkansas,"808,558","82,513","123,326","109,924","129,888","175,545","129,125","47,983","10,254"
8,California,"7,565,502","193,507","136,690","103,683","110,071","331,993","1,297,381","3,253,409","2,138,768"


+--------------------+---------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|               State|   Total:|Less than $50,000|$50,000 to $99,999|$100,000 to $149,999|$150,000 to $199,999|$200,000 to $299,999|$300,000 to $499,999|$500,000 to $999,999|$1,000,000 or more|
+--------------------+---------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|             Alabama|1,416,333|          147,494|           186,145|             179,651|             192,181|             298,254|             278,735|             115,476|            18,397|
|              Alaska|  181,586|           10,015|             6,103|               7,964|              11,081|              38,637|              75,646|              29,463|             2,677|
|             Arizona|1,921,74

Let's now incorporate the **last table from ACS** titled "Income in the Past 12 Months (in 2022 Inflation-Adjusted Dollars)" from the American Community Survey (ACS). This dataset provides insights into household incomes across different states, which can be a valuable factor in analyzing housing prices.

In [13]:
household_income_df = pd.read_csv("household_income.csv")
household_income_df

metadata_df = pd.read_csv("household_income_metadata.csv")

# Apply the column renaming directly to the original data_df (excluding the first header row in data_df)
household_income_df.columns = household_income_df.iloc[0]  # Use the first row as column headers
household_income_df = household_income_df[1:]  # Drop the first row now that it's been used for column names

# Since the first row of data_df is now the column names, we need to adjust the column names based on the metadata
# Creating a new mapping since the original data_df had its columns replaced by the first row
new_column_mapping = {code: label for code, label in zip(household_income_df.columns, metadata_df['Label'])}

# Rename the columns in data_df using the new mapping
household_income_df.rename(columns=new_column_mapping, inplace=True)

# Display the first few rows of the now correctly named data_df to confirm
# Reset the indexes 
household_income_df.reset_index(drop=True, inplace=True)
household_income_df.head(10)

# Convert the Pandas DataFrame to a Spark DataFrame
household_income_spark = spark.createDataFrame(household_income_df)

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, DecimalType

household_income_spark = household_income_spark.select(
    col("Geographic Area Name").alias("state").cast(StringType()),
    col("Geography").alias("geo_id").cast(StringType()),
    col("Estimate!!Households!!Total").alias("total_households").cast(IntegerType()),
    col("Estimate!!Households!!Total!!Less than $10,000").alias("less_than_10000").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$10,000 to $14,999").alias("between_10000_and_14999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$15,000 to $24,999").alias("between_15000_and_24999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$25,000 to $34,999").alias("between_25000_and_34999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$35,000 to $49,999").alias("between_35000_and_49999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$50,000 to $74,999").alias("between_50000_and_74999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$75,000 to $99,999").alias("between_75000_and_99999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Total!!$100,000 to $149,999").alias("between_100000_and_149999").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Median income (dollars)").alias("median_income").cast(DecimalType(10, 2)),
    col("Estimate!!Households!!Mean income (dollars)").alias("mean_income").cast(DecimalType(10, 2)),
)

household_income_spark.show()

# Write the Spark DataFrame to the PostgreSQL database
url = f"jdbc:postgresql://{host}:{port}/{dbname}?currentSchema={schema}"
household_income_spark.write.jdbc(url=url, table="household_income", mode='overwrite', properties=properties)
# Connection properties
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}


,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C01_002E,S1901_C01_002M,S1901_C01_003E,S1901_C01_003M,S1901_C01_004E,S1901_C01_004M,...,S1901_C04_012M,S1901_C04_013E,S1901_C04_013M,S1901_C04_014E,S1901_C04_014M,S1901_C04_015E,S1901_C04_015M,S1901_C04_016E,S1901_C04_016M,Unnamed: 130
0,Geography,Geographic Area Name,Estimate!!Households!!Total,Margin of Error!!Households!!Total,"Estimate!!Households!!Total!!Less than $10,000",Margin of Error!!Households!!Total!!Less than ...,"Estimate!!Households!!Total!!$10,000 to $14,999","Margin of Error!!Households!!Total!!$10,000 to...","Estimate!!Households!!Total!!$15,000 to $24,999","Margin of Error!!Households!!Total!!$15,000 to...",...,Margin of Error!!Nonfamily households!!Median ...,Estimate!!Nonfamily households!!Mean income (d...,Margin of Error!!Nonfamily households!!Mean in...,Estimate!!Nonfamily households!!PERCENT ALLOCA...,Margin of Error!!Nonfamily households!!PERCENT...,Estimate!!Nonfamily households!!PERCENT ALLOCA...,Margin of Error!!Nonfamily households!!PERCENT...,Estimate!!Nonfamily households!!PERCENT ALLOCA...,Margin of Error!!Nonfamily households!!PERCENT...,NaN
1,0400000US01,Alabama,2016448,11475,7.6,0.3,5.2,0.3,9.3,0.3,...,919,47145,1778,(X),(X),(X),(X),34.2,(X),NaN
2,0400000US02,Alaska,274574,3261,4.4,0.7,2.5,0.4,5.4,0.8,...,5211,74242,4733,(X),(X),(X),(X),30.3,(X),NaN
3,0400000US04,Arizona,2850377,11519,5.4,0.3,3.0,0.2,6.2,0.3,...,1067,66724,1965,(X),(X),(X),(X),33.6,(X),NaN
4,0400000US05,Arkansas,1216207,8435,7.3,0.5,5.9,0.4,9.5,0.4,...,773,45384,1596,(X),(X),(X),(X),34.4,(X),NaN
5,0400000US06,California,13550586,19485,4.8,0.1,3.2,0.1,5.5,0.1,...,751,89374,995,(X),(X),(X),(X),33.3,(X),NaN
6,0400000US08,Colorado,2384584,7807,4.1,0.2,2.5,0.2,5.4,0.3,...,1653,77519,1515,(X),(X),(X),(X),30.8,(X),NaN
7,0400000US09,Connecticut,1433635,7120,5.1,0.4,3.2,0.3,5.9,0.4,...,1374,72770,2549,(X),(X),(X),(X),33.8,(X),NaN
8,0400000US10,Delaware,402334,3985,4.0,0.6,2.6,0.5,5.8,0.6,...,2718,62657,2400,(X),(X),(X),(X),29.7,(X),NaN
9,0400000US11,District of Columbia,326970,3481,6.8,1.0,2.8,0.6,5.1,1.0,...,3796,112447,5167,(X),(X),(X),(X),24.0,(X),NaN


/tmp/ipykernel_132/122886211.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_income_df.rename(columns=new_column_mapping, inplace=True)


,Geography,Geographic Area Name,Estimate!!Households!!Total,Margin of Error!!Households!!Total,"Estimate!!Households!!Total!!Less than $10,000","Margin of Error!!Households!!Total!!Less than $10,000","Estimate!!Households!!Total!!$10,000 to $14,999","Margin of Error!!Households!!Total!!$10,000 to $14,999","Estimate!!Households!!Total!!$15,000 to $24,999","Margin of Error!!Households!!Total!!$15,000 to $24,999",...,Margin of Error!!Nonfamily households!!Median income (dollars),Estimate!!Nonfamily households!!Mean income (dollars),Margin of Error!!Nonfamily households!!Mean income (dollars),Estimate!!Nonfamily households!!PERCENT ALLOCATED!!Household income in the past 12 months,Margin of Error!!Nonfamily households!!PERCENT ALLOCATED!!Household income in the past 12 months,Estimate!!Nonfamily households!!PERCENT ALLOCATED!!Family income in the past 12 months,Margin of Error!!Nonfamily households!!PERCENT ALLOCATED!!Family income in the past 12 months,Estimate!!Nonfamily households!!PERCENT ALLOCATED!!Nonfamily income in the past 12 months,Margin of Error!!Nonfamily households!!PERCENT ALLOCATED!!Nonfamily income in the past 12 months,NaN
0,0400000US01,Alabama,2016448,11475,7.6,0.3,5.2,0.3,9.3,0.3,...,919,47145,1778,(X),(X),(X),(X),34.2,(X),NaN
1,0400000US02,Alaska,274574,3261,4.4,0.7,2.5,0.4,5.4,0.8,...,5211,74242,4733,(X),(X),(X),(X),30.3,(X),NaN
2,0400000US04,Arizona,2850377,11519,5.4,0.3,3.0,0.2,6.2,0.3,...,1067,66724,1965,(X),(X),(X),(X),33.6,(X),NaN
3,0400000US05,Arkansas,1216207,8435,7.3,0.5,5.9,0.4,9.5,0.4,...,773,45384,1596,(X),(X),(X),(X),34.4,(X),NaN
4,0400000US06,California,13550586,19485,4.8,0.1,3.2,0.1,5.5,0.1,...,751,89374,995,(X),(X),(X),(X),33.3,(X),NaN
5,0400000US08,Colorado,2384584,7807,4.1,0.2,2.5,0.2,5.4,0.3,...,1653,77519,1515,(X),(X),(X),(X),30.8,(X),NaN
6,0400000US09,Connecticut,1433635,7120,5.1,0.4,3.2,0.3,5.9,0.4,...,1374,72770,2549,(X),(X),(X),(X),33.8,(X),NaN
7,0400000US10,Delaware,402334,3985,4.0,0.6,2.6,0.5,5.8,0.6,...,2718,62657,2400,(X),(X),(X),(X),29.7,(X),NaN
8,0400000US11,District of Columbia,326970,3481,6.8,1.0,2.8,0.6,5.1,1.0,...,3796,112447,5167,(X),(X),(X),(X),24.0,(X),NaN
9,0400000US12,Florida,8826394,23013,5.6,0.2,3.5,0.1,7.2,0.2,...,612,64627,1040,(X),(X),(X),(X),33.1,(X),NaN


24/03/09 11:54:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------+----------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-------------------------+-------------+-----------+
|               state|     geo_id|total_households|less_than_10000|between_10000_and_14999|between_15000_and_24999|between_25000_and_34999|between_35000_and_49999|between_50000_and_74999|between_75000_and_99999|between_100000_and_149999|median_income|mean_income|
+--------------------+-----------+----------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-------------------------+-------------+-----------+
|             Alabama|0400000US01|         2016448|           7.60|                   5.20|                   9.30|                   8.60|                  12.00|                  16.70|                  12.

Finally let's print out **all existing tables** in our Postgres database.

In [14]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password, port=port)
# Create Cursor object 
cur = conn.cursor()

# Query to fetch table names in the specified schema
cur.execute(f"""
SELECT table_name FROM information_schema.tables
WHERE table_schema = '{schema}' AND table_type = 'BASE TABLE';
""")

# Fetch all table names
tables = cur.fetchall()

# Iterate over table names and select data from each table
for table in tables:
    table_name = table[0]
    print(f"Data from table: {table_name}")
    
    # Fetch data from each table
    query = f"SELECT * FROM {schema}.{table_name};"
    df = pd.read_sql_query(query, conn)
    
    # Display the DataFrame
    print(df.head())  # Adjust as per your need
    
# Close the cursor and connection
cur.close()
conn.close()


Data from table: us_eco
         date  gdp_value  unemployment_rate  inflation_rate  inclome
0  1948-07-01    279.196                3.6             NaN      NaN
1  1983-07-01   3689.179                9.4             NaN   5647.0
2  1987-10-01   5007.994                6.0             NaN   6619.6
3  1979-04-01   2591.247                5.8             NaN   5156.1
4  1980-07-01   2856.483                7.8             NaN   5189.8
Data from table: people
   id      name      city profession
0   1  John Doe  New York   Engineer
1   2  John Doe  New York   Engineer
2  34  John Doe  New York   Engineer
3  35  John Doe  New York   Engineer
4  36  John Doe  New York   Engineer
Data from table: us_estate
         date      cpi  permits  home_sales  price_index
0  1995-12-01  159.600   1442.0         NaN          NaN
1  1999-02-01  175.500   1720.0         NaN          NaN
2  2016-03-01  293.471   1118.0         NaN          NaN
3  1983-07-01  100.200   1795.0         NaN          NaN
4  2

/tmp/ipykernel_132/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_132/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_132/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_132/1827585956.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other

In [15]:
import psycopg2
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Connect to the PostgreSQL database
conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password, port=port)

# output table 'household_income'
# I assume you have a specific table name to use, replacing 'tables' loop since it's not defined in your snippet
table_name = 'household_income'  # Replace 'household_income' with your actual table name if different
print(f"Data from table: {table_name}")

# Fetch data from table
query = f"SELECT * FROM {schema}.{table_name};"  # Ensure 'schema' is defined or replace with actual schema name
df = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df.head())  # Adjust as per your need

# Close the connection
conn.close()

def convert_df_to_parquet(df, file_path):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, file_path)

# Convert and save as Parquet files
# Ensure 'df' is the DataFrame you want to convert, here it's from the 'household_income' table
convert_df_to_parquet(df, 'household_income.parquet')


Data from table: household_income
        state       geo_id  total_households  less_than_10000  \
0     Alabama  0400000US01           2016448              7.6   
1      Alaska  0400000US02            274574              4.4   
2     Arizona  0400000US04           2850377              5.4   
3    Arkansas  0400000US05           1216207              7.3   
4  California  0400000US06          13550586              4.8   

   between_10000_and_14999  between_15000_and_24999  between_25000_and_34999  \
0                      5.2                      9.3                      8.6   
1                      2.5                      5.4                      5.3   
2                      3.0                      6.2                      7.3   
3                      5.9                      9.5                      9.1   
4                      3.2                      5.5                      5.9   

   between_35000_and_49999  between_50000_and_74999  between_75000_and_99999  \
0             

/tmp/ipykernel_132/1686154386.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


After we have done uplaoding, and cleaning tables in the Postgres, let's now move our data into a **OLAP database--DuckDB**, preparing for future analysis. First le'ts get connected to DuckDB.

In [16]:
# To use the latest version of duckdb
!pip install duckdb

# Assuming you have DuckDB installed and the duckdb Python package
import duckdb

# Connect to DuckDB
conn = duckdb.connect()

  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/99/61/1ef0d3d13908b5ae6637103e0dbeafc58028f662caa366ecd871154dbc9f/duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (763 bytes)
Using cached duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.8 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Next, we will create a schema, then generate tables in the schema to store the data we sourced from the ACS website. Let's first create a tabe called **QuantileValue**, which contains the upper, median and lower value across all U.S. states. 

In [17]:
# Create schema
conn.execute("CREATE SCHEMA IF NOT EXISTS US_real_estate_market")

# Save the combined DataFrame as a Parquet file
combined_quartile_df.write.parquet("combined_value_quartile.parquet", mode="overwrite")

# SQL statement to create the quanitle table
create_table_query = '''
CREATE TABLE IF NOT EXISTS QuantileValue (
    State VARCHAR,
    UpperValue DECIMAL(10,2),
    MedianValue DECIMAL(10,2),
    LowerValue DECIMAL(10,2)
);
'''

# Execute the SQL statement to create the table
conn.execute(create_table_query)

# Verify the table creation
tables = conn.execute("SHOW TABLES").fetchall()
print(tables)

24/03/09 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 11:54:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/09 1

[('QuantileValue',)]


After checking that we have successfully created QuantileValue Table, let's create another table called **NumOfHousese**, which corresponds to one of our ACS table. 

In [18]:
# SQL statement to create the num_of_houses table based on the structure in the provided image
create_table_query = '''
CREATE TABLE IF NOT EXISTS NumOfHouses (
    State VARCHAR,
    Total INTEGER,
    Less_than_50000 INTEGER,
    From_50000_to_99999 INTEGER,
    From_100000_to_149999 INTEGER,
    From_150000_to_199999 INTEGER,
    From_200000_to_299999 INTEGER,
    From_300000_to_499999 INTEGER,
    From_500000_to_999999 INTEGER,
    One_million_or_more INTEGER
);
'''

# Execute the SQL statement to create the table
conn.execute(create_table_query)

# Verify the table creation
tables = conn.execute("SHOW TABLES").fetchall()
print("Tables in the database:", tables)


Tables in the database: [('NumOfHouses',), ('QuantileValue',)]
